https://github.com/eriklindernoren/PyTorch-GAN/blob/master/implementations/lsgan/lsgan.py

In [1]:
from pathlib import Path
USE_COLAB: bool = True
dataset_base_path = Path("/content/drive/My Drive/ECE 792 - Advance Topics in Machine Learning/Datasets/RealFaces/CelebA")
if USE_COLAB:
  from google.colab import drive
  
  # Mount the drive to access google shared docs
  drive.mount('/content/drive/', force_remount=True)

  if dataset_base_path.exists():
    print("Folder exists")
  else:
    print("DOESN'T EXIST. Add desired folder as a shortcut in your 'My Drive'")

Mounted at /content/drive/
Folder exists


In [2]:
import numpy as np
import math
import torchvision.transforms as transforms
from torchvision.utils import save_image

from torch.utils.data import DataLoader
from torchvision import datasets
from torch.autograd import Variable

import torch.nn as nn
import torch.nn.functional as F
import torch

from zipfile import ZipFile
from PIL import Image

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
class CelebrityData(torch.utils.data.Dataset):

  def __init__(self,zip_file,transform = None) :
    self.zip_file = zip_file
    self.transform = transform

    # Create a ZipFile Object and load sample.zip in it
    with ZipFile(str(zip_file), 'r') as zipObj:
      # Extract all the contents of zip file in current directory
      zipObj.extractall()


    self.ListOfImages = zipObj.namelist()[1:]


  def __getitem__(self,index):

    image = Image.open(self.ListOfImages[index]).convert('RGB')

    if self.transform is not None:
      image = self.transform(image)

    return image


  def __len__(self):
    return  len(self.ListOfImages)

In [4]:
def weights_init_normal(m):
  classname = m.__class__.__name__
  if classname.find("Conv") != -1:
    torch.nn.init.normal_(m.weight.data, 0.0, 0.02)
  elif classname.find("BatchNorm") != -1:
    torch.nn.init.normal_(m.weight.data, 1.0, 0.02)
    torch.nn.init.constant_(m.bias.data, 0.0)

In [5]:
output_dir = "/content/drive/My Drive/ECE 792 - Advance Topics in Machine Learning/Code/DatasetGeneration/LSGAN/models"
Path(output_dir).mkdir(exist_ok=True, parents=True)
zip_file = dataset_base_path.parent / "img_align_celeb.zip"
gen_imgs_path = Path(output_dir) / "images"
gen_imgs_path.mkdir(exist_ok=True, parents=True)

config = {
    "n_epochs": 50,
    "batch_size": 64,
    "lr": 2e-4,
    "b1": 0.5,
    "b2": 0.999,
    "latent_dim": 100,
    "img_size": 64,
    "n_channels": 3,
    "chkp_freq": 1,
    "n_sampled_imgs": 25,
}

In [6]:
class Generator(nn.Module):
  def __init__(self, img_size: int = 64, latent_dim: int = 100, n_channels: int = 3):
    super(Generator, self).__init__()

    self.init_size = img_size // 4
    self.l1 = nn.Sequential(nn.Linear(latent_dim, 128 * self.init_size ** 2))

    self.conv_blocks = nn.Sequential(
        nn.Upsample(scale_factor=2),
        nn.Conv2d(in_channels=128, out_channels=128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
        nn.BatchNorm2d(128, 0.8),
        nn.LeakyReLU(0.2, inplace=True),
        nn.Upsample(scale_factor=2),
        nn.Conv2d(in_channels=128, out_channels=64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
        nn.BatchNorm2d(64, 0.8),
        nn.LeakyReLU(0.2, inplace=True),
        nn.Conv2d(in_channels=64, out_channels=n_channels, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
        nn.Tanh(),
    )

  def forward(self, z):
    out = self.l1(z)
    out = out.view(out.shape[0], 128, self.init_size, self.init_size)
    img = self.conv_blocks(out)

    return img


class Discriminator(nn.Module):
  def __init__(self, img_size: int = 64, n_channels: int = 3):
    super(Discriminator, self).__init__()

    def discriminator_block(in_filters, out_filters, bn=True):
      block = [nn.Conv2d(in_filters, out_filters, 3, 2, 1), nn.LeakyReLU(0.2, inplace=True), nn.Dropout2d(0.25)]
      if bn:
        block.append(nn.BatchNorm2d(out_filters, 0.8))
      
      return block
    
    self.model = nn.Sequential(
        *discriminator_block(n_channels, 16, bn=False),
        *discriminator_block(16, 32),
        *discriminator_block(32, 64),
        *discriminator_block(64, 128),
    )

    ds = img_size // 2 ** 4
    self.adv_layer = nn.Linear(128 * ds ** 2, 1)

  def forward(self, img):
    out = self.model(img)
    out = out.view(out.shape[0], -1)
    validity = self.adv_layer(out)

    return validity

In [7]:
# Note that multiple data augmentation techniques are used to learn generalize better, 

CelebrityData = CelebrityData(zip_file = zip_file,
                              transform = transforms.Compose([
                                transforms.Resize(int(config['img_size'] * 1.1)),
                                transforms.CenterCrop(config['img_size']),
                                transforms.ToTensor(),
                                transforms.Normalize(mean = (0.5,0.5,0.5),
                                                     std  = (0.5,0.5,0.5))       
                                
]))

dataloader = torch.utils.data.DataLoader(dataset = CelebrityData,
                                         shuffle = True,
                                         batch_size = config['batch_size'],
                                         num_workers = 2,
                                         drop_last = True,
                                         pin_memory = True) 

In [8]:
adversarial_loss = torch.nn.MSELoss().to(device)
generator = Generator(config["img_size"], config["latent_dim"], config["n_channels"]).to(device)
generator.apply(weights_init_normal)
discriminator = Discriminator(config["img_size"], config["n_channels"]).to(device)
discriminator.apply(weights_init_normal)

print(f"generator: {list(generator.state_dict().keys())}\n\ndiscriminator: {list(discriminator.state_dict().keys())}")

generator: ['l1.0.weight', 'l1.0.bias', 'conv_blocks.1.weight', 'conv_blocks.1.bias', 'conv_blocks.2.weight', 'conv_blocks.2.bias', 'conv_blocks.2.running_mean', 'conv_blocks.2.running_var', 'conv_blocks.2.num_batches_tracked', 'conv_blocks.5.weight', 'conv_blocks.5.bias', 'conv_blocks.6.weight', 'conv_blocks.6.bias', 'conv_blocks.6.running_mean', 'conv_blocks.6.running_var', 'conv_blocks.6.num_batches_tracked', 'conv_blocks.8.weight', 'conv_blocks.8.bias']

discriminator: ['model.0.weight', 'model.0.bias', 'model.3.weight', 'model.3.bias', 'model.6.weight', 'model.6.bias', 'model.6.running_mean', 'model.6.running_var', 'model.6.num_batches_tracked', 'model.7.weight', 'model.7.bias', 'model.10.weight', 'model.10.bias', 'model.10.running_mean', 'model.10.running_var', 'model.10.num_batches_tracked', 'model.11.weight', 'model.11.bias', 'model.14.weight', 'model.14.bias', 'model.14.running_mean', 'model.14.running_var', 'model.14.num_batches_tracked', 'adv_layer.weight', 'adv_layer.bias']

In [9]:
from tqdm import tqdm

optimizer_G = torch.optim.Adam(generator.parameters(), lr=config["lr"], betas=(config["b1"], config["b2"]))
optimizer_D = torch.optim.Adam(discriminator.parameters(), lr=config["lr"], betas=(config["b1"], config["b2"]))

fixed_noise = torch.randn(config["n_sampled_imgs"], config["latent_dim"]).to(device)
print("Starting Training Loop...")
epoch_tqdm = tqdm(total=config["n_epochs"], position=0)
for epoch in range(config["n_epochs"]):
  model_output_path = Path(output_dir) / f"LSGAN--{epoch+1}.pth"
  epoch_gen_imgs_path = gen_imgs_path / f"{epoch+1}"
  epoch_gen_imgs_path.mkdir(exist_ok=True, parents=True)

  for batch_idx, imgs in enumerate(dataloader):
    # adversarial ground truths
    imgs = imgs.to(device)
    valid = Variable(torch.Tensor(config["batch_size"], 1).fill_(1.0), requires_grad=False).to(device)
    fake = Variable(torch.Tensor(config["batch_size"], 1).fill_(0.0), requires_grad=False).to(device)

    # Configure input
    real_imgs = Variable(imgs)

    # Train Discriminator
    optimizer_D.zero_grad()

    # generate images from generator
    # sample noise as generator input
    z = Variable(torch.randn(config["batch_size"], config["latent_dim"]).to(device))
    gen_imgs = generator(z).detach()

    # Measure the discriminator's ability to classify real from generated samples
    real_loss = adversarial_loss(discriminator(real_imgs), valid)
    fake_loss = adversarial_loss(discriminator(gen_imgs), fake)
    d_loss = 0.5 * (real_loss + fake_loss)

    d_loss.backward()
    optimizer_D.step()

    # Train Generator
    optimizer_G.zero_grad()

    # generate a batch of images using same noise for discriminator training
    gen_imgs = generator(z)

    # Loss measures generator's ability to fool the discriminator
    g_loss = adversarial_loss(discriminator(gen_imgs), valid)

    g_loss.backward()
    optimizer_G.step()

    if (batch_idx + 1) % 50 == 0:
      print(" [%d/%d] [%d/%d]\tLoss_D: %.8f\tLoss_G: %.8f" % (
          epoch, config["n_epochs"], batch_idx + 1, len(dataloader), d_loss.item(), g_loss.item()
      ))

  epoch_tqdm.update(1)
  if (epoch + 1) % config["chkp_freq"] == 0:
    torch.save(
        {
            "Generator": generator.state_dict(),
            "Discriminator": discriminator.state_dict(),
        },
        str(model_output_path)
    )

    generator.eval()
    generated_imgs = generator(fixed_noise)
    for idx, gen_img in enumerate(generated_imgs):
      img_output_path = epoch_gen_imgs_path / f"{idx}.png"
      save_image(gen_img, img_output_path)

    generator.train()

Starting Training Loop...


  0%|          | 0/3165 [00:00<?, ?it/s]

 [0/50] [50/3165]	Loss_D: 0.25953257	Loss_G: 0.32663801
 [0/50] [100/3165]	Loss_D: 0.24879390	Loss_G: 0.29366833
 [0/50] [150/3165]	Loss_D: 0.24053468	Loss_G: 0.25317475
 [0/50] [200/3165]	Loss_D: 0.25294295	Loss_G: 0.25885463
 [0/50] [250/3165]	Loss_D: 0.25806653	Loss_G: 0.24723428
 [0/50] [300/3165]	Loss_D: 0.25535476	Loss_G: 0.25212488
 [0/50] [350/3165]	Loss_D: 0.24737847	Loss_G: 0.24740274
 [0/50] [400/3165]	Loss_D: 0.24847053	Loss_G: 0.24883825
 [0/50] [450/3165]	Loss_D: 0.25311714	Loss_G: 0.25960603
 [0/50] [500/3165]	Loss_D: 0.25073656	Loss_G: 0.25177559
 [0/50] [550/3165]	Loss_D: 0.25465858	Loss_G: 0.24896106
 [0/50] [600/3165]	Loss_D: 0.25211450	Loss_G: 0.25435340
 [0/50] [650/3165]	Loss_D: 0.25101322	Loss_G: 0.25022697
 [0/50] [700/3165]	Loss_D: 0.24899882	Loss_G: 0.25016099
 [0/50] [750/3165]	Loss_D: 0.25225300	Loss_G: 0.25437057
 [0/50] [800/3165]	Loss_D: 0.25493053	Loss_G: 0.24849439
 [0/50] [850/3165]	Loss_D: 0.25026911	Loss_G: 0.25698853
 [0/50] [900/3165]	Loss_D: 0.241

  0%|          | 1/3165 [02:40<140:51:06, 160.26s/it]

 [1/50] [50/3165]	Loss_D: 0.24176538	Loss_G: 0.25844070
 [1/50] [100/3165]	Loss_D: 0.25410435	Loss_G: 0.25149950
 [1/50] [150/3165]	Loss_D: 0.25279635	Loss_G: 0.25651520
 [1/50] [200/3165]	Loss_D: 0.25522131	Loss_G: 0.24676931
 [1/50] [250/3165]	Loss_D: 0.24622911	Loss_G: 0.25319278
 [1/50] [300/3165]	Loss_D: 0.24538548	Loss_G: 0.25980121
 [1/50] [350/3165]	Loss_D: 0.26284707	Loss_G: 0.24373317
 [1/50] [400/3165]	Loss_D: 0.25242186	Loss_G: 0.24818489
 [1/50] [450/3165]	Loss_D: 0.25241160	Loss_G: 0.25809205
 [1/50] [500/3165]	Loss_D: 0.25183159	Loss_G: 0.25189078
 [1/50] [550/3165]	Loss_D: 0.25307187	Loss_G: 0.25304770
 [1/50] [600/3165]	Loss_D: 0.24418288	Loss_G: 0.25246161
 [1/50] [650/3165]	Loss_D: 0.25022820	Loss_G: 0.25625563
 [1/50] [700/3165]	Loss_D: 0.24747628	Loss_G: 0.25365686
 [1/50] [750/3165]	Loss_D: 0.24475873	Loss_G: 0.24818707
 [1/50] [800/3165]	Loss_D: 0.25564623	Loss_G: 0.25808990
 [1/50] [850/3165]	Loss_D: 0.24971975	Loss_G: 0.26306099
 [1/50] [900/3165]	Loss_D: 0.246

...

  2%|▏         | 48/3165 [2:01:06<131:13:30, 151.56s/it]

 [48/50] [50/3165]	Loss_D: 0.27686188	Loss_G: 0.29221725
 [48/50] [100/3165]	Loss_D: 0.21169814	Loss_G: 0.23360337
 [48/50] [150/3165]	Loss_D: 0.23729607	Loss_G: 0.22386375
 [48/50] [200/3165]	Loss_D: 0.22264121	Loss_G: 0.32285291
 [48/50] [250/3165]	Loss_D: 0.25899389	Loss_G: 0.28533015
 [48/50] [300/3165]	Loss_D: 0.23454061	Loss_G: 0.36458763
 [48/50] [350/3165]	Loss_D: 0.20069528	Loss_G: 0.27164131
 [48/50] [400/3165]	Loss_D: 0.20552719	Loss_G: 0.24167040
 [48/50] [450/3165]	Loss_D: 0.28481159	Loss_G: 0.25776026
 [48/50] [500/3165]	Loss_D: 0.24170224	Loss_G: 0.42762023
 [48/50] [550/3165]	Loss_D: 0.28745168	Loss_G: 0.34730077
 [48/50] [600/3165]	Loss_D: 0.22519311	Loss_G: 0.36270052
 [48/50] [650/3165]	Loss_D: 0.25040907	Loss_G: 0.25217980
 [48/50] [700/3165]	Loss_D: 0.19172478	Loss_G: 0.22462213
 [48/50] [750/3165]	Loss_D: 0.28032023	Loss_G: 0.27370083
 [48/50] [800/3165]	Loss_D: 0.26416150	Loss_G: 0.27176273
 [48/50] [850/3165]	Loss_D: 0.22132419	Loss_G: 0.35353726
 [48/50] [900/3

  2%|▏         | 49/3165 [2:03:36<130:54:48, 151.25s/it]

 [49/50] [50/3165]	Loss_D: 0.20789182	Loss_G: 0.28217494
 [49/50] [100/3165]	Loss_D: 0.23466787	Loss_G: 0.28706756
 [49/50] [150/3165]	Loss_D: 0.23034447	Loss_G: 0.30548406
 [49/50] [200/3165]	Loss_D: 0.22147623	Loss_G: 0.32943189
 [49/50] [250/3165]	Loss_D: 0.26409751	Loss_G: 0.31662589
 [49/50] [300/3165]	Loss_D: 0.26181376	Loss_G: 0.23294055
 [49/50] [350/3165]	Loss_D: 0.22542027	Loss_G: 0.27770787
 [49/50] [400/3165]	Loss_D: 0.20879805	Loss_G: 0.32006323
 [49/50] [450/3165]	Loss_D: 0.23245989	Loss_G: 0.35667467
 [49/50] [500/3165]	Loss_D: 0.23351185	Loss_G: 0.38095111
 [49/50] [550/3165]	Loss_D: 0.26647156	Loss_G: 0.34907469
 [49/50] [600/3165]	Loss_D: 0.19720253	Loss_G: 0.27318084
 [49/50] [650/3165]	Loss_D: 0.23369883	Loss_G: 0.32122684
 [49/50] [700/3165]	Loss_D: 0.23491628	Loss_G: 0.24568716
 [49/50] [750/3165]	Loss_D: 0.25178632	Loss_G: 0.25594217
 [49/50] [800/3165]	Loss_D: 0.21347958	Loss_G: 0.24378854
 [49/50] [850/3165]	Loss_D: 0.29803935	Loss_G: 0.30890825
 [49/50] [900/3

  2%|▏         | 50/3165 [2:06:08<130:54:12, 151.29s/it]